<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/bert-base-uncased-paradetox-with-labels-sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install transformers[torch]
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/paradetox_with_labels")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/39488 [00:00<?, ? examples/s]

In [3]:
'''
Data Preprocessing

# Split the toxic and neutral comments and assign labels
toxic_data = dataset['train']['en_toxic_comment']
neutral_data = dataset['train']['en_neutral_comment']

toxic_labels = [0] * len(toxic_data)
neutral_labels = [1] * len(neutral_data)

# Combine the data and labels
all_texts = toxic_data + neutral_data
all_labels = toxic_labels + neutral_labels

# Create a new dataset with the combined data
from datasets import Dataset

formatted_dataset = Dataset.from_dict({
    'text': all_texts,
    'label': all_labels
})

# Shuffle the dataset for randomness
formatted_dataset = formatted_dataset.shuffle()

formatted_dataset.push_to_hub("paradetox_with_labels")
'''

'\nData Preprocessing\n\n# Split the toxic and neutral comments and assign labels\ntoxic_data = dataset[\'train\'][\'en_toxic_comment\']\nneutral_data = dataset[\'train\'][\'en_neutral_comment\']\n\ntoxic_labels = [0] * len(toxic_data)\nneutral_labels = [1] * len(neutral_data)\n\n# Combine the data and labels\nall_texts = toxic_data + neutral_data\nall_labels = toxic_labels + neutral_labels\n\n# Create a new dataset with the combined data\nfrom datasets import Dataset\n\nformatted_dataset = Dataset.from_dict({\n    \'text\': all_texts,\n    \'label\': all_labels\n})\n\n# Shuffle the dataset for randomness\nformatted_dataset = formatted_dataset.shuffle()\n\nformatted_dataset.push_to_hub("paradetox_with_labels")\n'

In [5]:
# Tokenization
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/39488 [00:00<?, ? examples/s]

In [6]:
# Model Selection
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Training Arguments
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    do_train=True,
    learning_rate=2e-5,
    output_dir='bert-base-uncased-paradetox_with_labels',
    overwrite_output_dir=True,
    push_to_hub=True,
    save_steps=10_000,
    save_total_limit=2,
)

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39488
    })
})

In [12]:
# Splitting dataset
from datasets import DatasetDict

split = tokenized_datasets["train"].train_test_split(test_size=0.1)
dataset_split = DatasetDict({"train": split["train"], "test": split["test"]})

In [14]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_split["train"],
    eval_dataset=dataset_split["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.085500,0.070107
2,0.041300,0.089699
3,0.027500,0.092243


TrainOutput(global_step=13329, training_loss=0.0690386761213263, metrics={'train_runtime': 2692.6068, 'train_samples_per_second': 39.596, 'train_steps_per_second': 4.95, 'total_flos': 7013027847329280.0, 'train_loss': 0.0690386761213263, 'epoch': 3.0})

In [15]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/HamdanXI/bert-base-uncased-paradetox_with_labels/tree/main/'